## Purchasing Power Growth Depends on Industry
###### Changes since 2000 in the ability of workers to purchase goods and services varies by their industry of employment


*Brian Dew* <br>
*June 14, 2017*

-----

Notes:

In [3]:
# Import preliminaries
import requests
import json
import pandas as pd
import config
from collections import OrderedDict

In [4]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}
series_dict = {
    'Manufacturing': 'CES3000000008',
    'Mining and Logging': 'CES1000000008',
    'Information': 'CES5000000008',
    'Financial Activities': 'CES5500000008',
    'Professional and Business Services': 'CES6000000008',
    'Education and Health Services': 'CES6500000008',
    '<b>Total Private</b>': 'CES0500000008',
    'Construction': 'CES2000000008',
    'Food Services and Drinking Places': 'CES7072200008',
    'Transportation and Warehousing': 'CES4300000008',
    'Retail Trade': 'CES4200000008',
    'CPI-Urban Wage Earners and Clerical Workers': 'CWSR0000SA0',
}
inv_map = {v: k for k, v in series_dict.iteritems()}
series_list = series_dict.values()  # Take just the series codes as a list

date_list = [("1999", "2008"), ("2009", "2017")]
df = pd.DataFrame()

In [8]:
for start, end in date_list:
    data = json.dumps({"seriesid":series_list, "startyear":start, "endyear":end})
    json_data = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
    # Store the relevant information in a dictionary and combine into a pandas dataframe
    d = {}
    for series in json_data['Results']['series']:
        seriesId = series['seriesID']
        d[seriesId] = pd.DataFrame(series['data'])
        # Convert BLS API dates to readable format (YYYY-MM-DD)
        d[seriesId]['date'] = pd.to_datetime(d[seriesId]['period'] + ' ' + d[seriesId]['year'])
        # Keep only date and series values
        d[seriesId] = d[seriesId].sort_values(by='date')[['date', 'value']].set_index('date')
    # Rename and round
        d[seriesId] = d[seriesId]['value'].rename(inv_map[seriesId]).astype(float)
    df = df.append(pd.concat([d[series_list[x]] for x in range(len(d))], axis=1))

In [10]:
p_list = ['CPIAUCSL', 'CWUR0000SA0', 'CWSR0000SA0']  # Skip prices data
w_list = {k:v for k,v in series_dict.iteritems() if v not in p_list}
for k, v in w_list.iteritems():
    df[v+'_rw'] = df[k].multiply(df['CPI-Urban Wage Earners and Clerical Workers'][0]).divide(
        df['CPI-Urban Wage Earners and Clerical Workers'])
    df[v+'_rc'] = df[v+'_rw'].pct_change(12).multiply(100)
df = df.dropna()

In [12]:
for k, v in w_list.iteritems():
    w_list[k] = {'series': v}
    w_list[k]['current_month'] = df.index[-1].strftime('%b %Y')
    w_list[k]['nominal_wage'] = '${:.2f}'.format(df[k][-1])
    w_list[k]['wage_growth_curr'] = '{}%'.format(str(df[v+'_rc'][-1].round(2)))
    ch_val = df.loc['2000-01-01'][v+'_rw']
    w_list[k]['cumulative_change'] = round((df[v+'_rw'][-1] - ch_val)/ch_val * 100,2)
    w_list[k]['text'] = '<b>{}</b><br>Total Percent Change: <b>{:.1f}%</b><br>Average Hourly Earnings: <br><b>${:.2f}</b> (April 2017)'.format(
        k, w_list[k]['cumulative_change'], df[k][-1])
w_list = OrderedDict(sorted(w_list.iteritems(), key=lambda x: float(x[1]['cumulative_change'])))

In [13]:
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

trace1 = {
    "x": [float(w_list[k]['cumulative_change']) for k in w_list.keys()], 
    "y": w_list.keys(), 
    "text": [w_list[k]['text'] for k in w_list.keys()], 
    "name": "Total Change in Purchasing Power since 2000", 
    "orientation": "h", 
    "type": "bar",
    "hoverinfo": "text",
    "marker": {
        "color": "rgb(31,73,125)", 
        "line": {
          "color": "rgb(8,48,107)", 
          "width": 0
    }
  }, 
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": -0.65, 
      "y": 1.1432, 
      "align": "left", 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Worker purchasing power growth since January 2000 by industry</b><br><i>Production and non-supervisory employees, percent change, as of April 2017</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.65, 
      "y": -0.1554, 
      "align": "left", 
      "font": {"size": 11}, 
      "showarrow": False, 
      "text": "<a href=\"http://cepr.net>http://cepr.net</a><br>Source: Bureau of Labor Statistics Average Hourly Earnings, adjusted for inflation using CPI-W.", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": 22.2, 
      "y": 9.3, 
      "align": "left", 
      "arrowcolor": "rgba(0, 0, 0, 0)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": True, 
      "text": "24.6%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 17.4, 
      "y": "Professional and Business Services", 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "19.7%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 16.3, 
      "y": 8, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "18.7%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 15.1, 
      "y": 7, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "17.5%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 14.9, 
      "y": 6, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "17.4%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 11.8, 
      "y": 5, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "14.2%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 9.4, 
      "y": 4, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "<b>11.4%</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 5.5, 
      "y": 3, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(255, 255, 255)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "7.5%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 4.3, 
      "y": 2, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 0, 
      "ay": -20, 
      "font": {
        "color": "rgb(60, 60, 60)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "2.5%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 2.1, 
      "y": 1, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 25, 
      "ay": 0, 
      "font": {
        "color": "rgb(60, 60, 60)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "-0.3%", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 1.8, 
      "y": 0, 
      "align": "left", 
      "arrowcolor": "rgb(60, 60, 60)", 
      "arrowhead": 0, 
      "arrowwidth": 1, 
      "ax": 41, 
      "ay": -1, 
      "font": {
        "color": "rgb(60, 60, 60)", 
        "family": "\"Open Sans\", verdana, arial, sans-serif", 
        "size": 12
      }, 
      "showarrow": False, 
      "text": "-0.6%", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": False, 
  "height": 450, 
  "legend": {
    "x": 0.682337662338, 
    "y": 1
  }, 
  "margin": {
    "r": 20, 
    "t": 70, 
    "b": 60, 
    "l": 220
  }, 
  "showlegend": False, 
  "title": "", 
  "width": 550, 
  "xaxis": {
    "autorange": True, 
    "range": [-5, 30], 
    "type": "linear"
  }, 
  "yaxis": {
    "autorange": True, 
    "type": "category"
  }
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='Purchasing_Power_Growth_by_Industry')